# Immutable X Tax Reporter
This notebook will produce a CSV file compatible with Koinly, which can be modified for other tax report formats.
The resulting file may possibly contain errors, omissions or duplications, so please check the file manually before using it.

In [ ]:
import requests
import pandas as pd
from datetime import datetime as dt

In [ ]:
address = 'ACCOUNT_ADDRESS'

In [ ]:
url = 'https://api.x.immutable.com/v2/balances/' + address
response = requests.get(url)
tokens = {'': 'ETH'}
erc20 = ['ETH']
multipliers = {'ETH': 1e-18}

In [ ]:
while True:
    if response.json()['result'] == []:
        break
    for item in response.json()['result']:
        if item['symbol'] != 'ETH':
            tokens[item['token_address']] = item['symbol']
            erc20.append(item['symbol'])
            url = 'https://api.x.immutable.com/v1/tokens/' + item['token_address']
            resp = requests.get(url)
            multipliers[item['symbol']] = 10**(-(int(resp.json()['decimals'])))
    url = 'https://api.x.immutable.com/v2/balances/' + address + '&cursor=' + response.json()['cursor']
    response = requests.get(url)

In [ ]:
def tokenName(tokenAddress):
    global tokens, erc20, multipliers
    url = 'https://api.x.immutable.com/v1/tokens/' + tokenAddress
    response = requests.get(url)
    if 'name' in response.json():
        tokens[tokenAddress] = response.json()['name']
        erc20.append(response.json()['name'])
        multipliers[response.json()['name']] = 10**(-(int(resp.json()['decimals'])))
        return response.json()['name']
    else:
        url = 'https://api.x.immutable.com/v1/collections/' + tokenAddress
        response = requests.get(url)
        tokens[tokenAddress] = response.json()['name']
        multipliers[response.json()['name']] = 1
        return response.json()['name']

In [ ]:
report = pd.DataFrame(columns = ['Date', 'Sent Amount', 'Sent Currency', 'Received Amount', 'Received Currency',
                      'Fee Amount', 'Fee Currency', 'Description', 'TxHash'])

In [ ]:
url = 'https://api.x.immutable.com/v1/mints?user=' + address + '&status=success'
response = requests.get(url)
while True:
    if response.json()['result'] == []:
        break
    print("/", end="")
    for item in response.json()['result']:
        print(".", end="")
        txId = item['transaction_id']
        tokenAddress = item['token']['data']['token_address']
        if tokenAddress in tokens:
            token = tokens[tokenAddress]
        else:
            token = tokenName(tokenAddress)
        recAmt = int(item['token']['data']['quantity']) * multipliers[token]
        tstamp = item['timestamp']
        date = tstamp[8:10] + '/' + tstamp[5:7] + '/' + tstamp[0:4] + ' ' + tstamp[11:19]
        if token in erc20:
            report = report.append({'Date': date, 'Received Amount': recAmt, 'Received Currency': token,
                      'TxHash': txId}, ignore_index=True)
        else:
            report = report.append({'Date': date, 'Received Amount': recAmt, 'Received Currency': 'NULL',
                      'Description': token + ' #' + item['token']['data']['token_id'], 'TxHash': txId}, ignore_index=True)
    url = 'https://api.x.immutable.com/v1/mints?user=' + address + '&status=success&cursor=' + response.json()['cursor']
    response = requests.get(url)

In [ ]:
url = 'https://api.x.immutable.com/v1/transfers?user=' + address + '&status=success'
response = requests.get(url)
while True:
    if response.json()['result'] == []:
        break
    print("/", end="")
    for item in response.json()['result']:
        print(".", end="")
        txId = item['transaction_id']
        tokenAddress = item['token']['data']['token_address']
        if tokenAddress in tokens:
            token = tokens[tokenAddress]
        else:
            token = tokenName(tokenAddress)
        sentAmt = int(item['token']['data']['quantity']) * multipliers[token]
        tstamp = item['timestamp']
        date = tstamp[8:10] + '/' + tstamp[5:7] + '/' + tstamp[0:4] + ' ' + tstamp[11:19]
        if token in erc20:
            report = report.append({'Date': date, 'Sent Amount': sentAmt, 'Sent Currency': token,
                      'TxHash': txId}, ignore_index=True)
        else:
            report = report.append({'Date': date, 'Sent Amount': sentAmt, 'Sent Currency': 'NULL',
                      'Description': token + ' #' + item['token']['data']['token_id'], 'TxHash': txId}, ignore_index=True)
    url = 'https://api.x.immutable.com/v1/transfers?user=' + address + '&status=success&cursor=' + response.json()['cursor']
    response = requests.get(url)

In [ ]:
url = 'https://api.x.immutable.com/v1/transfers?receiver=' + address + '&status=success'
response = requests.get(url)
while True:
    if response.json()['result'] == []:
        break
    print("/", end="")
    for item in response.json()['result']:
        print(".", end="")
        txId = item['transaction_id']
        tokenAddress = item['token']['data']['token_address']
        if tokenAddress in tokens:
            token = tokens[tokenAddress]
        else:
            token = tokenName(tokenAddress)
        recAmt = int(item['token']['data']['quantity']) * multipliers[token]
        tstamp = item['timestamp']
        date = tstamp[8:10] + '/' + tstamp[5:7] + '/' + tstamp[0:4] + ' ' + tstamp[11:19]
        if token in erc20:
            report = report.append({'Date': date, 'Received Amount': recAmt, 'Received Currency': token,
                      'TxHash': txId}, ignore_index=True)
        else:
            report = report.append({'Date': date, 'Received Amount': recAmt, 'Received Currency': 'NULL',
                      'Description': token + ' #' + item['token']['data']['token_id'], 'TxHash': txId}, ignore_index=True)
    url = 'https://api.x.immutable.com/v1/transfers?receiver=' + address + '&status=success&cursor=' + response.json()['cursor']
    response = requests.get(url)

In [ ]:
url = 'https://api.x.immutable.com/v1/deposits?user=' + address + '&status=success'
response = requests.get(url)
while True:
    if response.json()['result'] == []:
        break
    print("/", end="")
    for item in response.json()['result']:
        print(".", end="")
        txId = item['transaction_id']
        tokenAddress = item['token']['data']['token_address']
        if tokenAddress in tokens:
            token = tokens[tokenAddress]
        else:
            token = tokenName(tokenAddress)
        recAmt = int(item['token']['data']['quantity']) * multipliers[token]
        tstamp = item['timestamp']
        date = tstamp[8:10] + '/' + tstamp[5:7] + '/' + tstamp[0:4] + ' ' + tstamp[11:19]
        if token in erc20:
            report = report.append({'Date': date, 'Received Amount': recAmt, 'Received Currency': token,
                      'TxHash': txId}, ignore_index=True)
        else:
            report = report.append({'Date': date, 'Received Amount': recAmt, 'Received Currency': 'NULL',
                      'Description': token + ' #' + item['token']['data']['token_id'], 'TxHash': txId}, ignore_index=True)
    url = 'https://api.x.immutable.com/v1/deposits?user=' + address + '&status=success&cursor=' + response.json()['cursor']
    response = requests.get(url)

In [ ]:
url = 'https://api.x.immutable.com/v1/withdrawals?user=' + address + '&status=success'
response = requests.get(url)
while True:
    if response.json()['result'] == []:
        break
    print("/", end="")
    for item in response.json()['result']:
        print(".", end="")
        txId = item['transaction_id']
        tokenAddress = item['token']['data']['token_address']
        if tokenAddress in tokens:
            token = tokens[tokenAddress]
        else:
            token = tokenName(tokenAddress)
        sentAmt = int(item['token']['data']['quantity']) * multipliers[token]
        tstamp = item['timestamp']
        date = tstamp[8:10] + '/' + tstamp[5:7] + '/' + tstamp[0:4] + ' ' + tstamp[11:19]
        if token in erc20:
            report = report.append({'Date': date, 'Sent Amount':sentAmt, 'Sent Currency': token,
                      'TxHash': txId}, ignore_index=True)
        else:
            report = report.append({'Date': date, 'Sent Amount': sentAmt, 'Received Currency': 'NULL',
                      'Description': token + ' #' + item['token']['data']['token_id'], 'TxHash': txId}, ignore_index=True)
    url = 'https://api.x.immutable.com/v1/withdrawals?user=' + address + '&status=success&cursor=' + response.json()['cursor']
    response = requests.get(url)

In [ ]:
url = 'https://api.x.immutable.com/v1/orders?user=' + address + '&status=filled'
response = requests.get(url)
while True:
    if response.json()['result'] == []:
        break
    print("/", end="")
    for item in response.json()['result']:
        print(".", end="")
        txId = item['order_id']
        # Outgoing token(s):
        tokenAddress = item['sell']['data']['token_address']
        if tokenAddress in tokens:
            sentToken = tokens[tokenAddress]
        else:
            sentToken = tokenName(tokenAddress)
        sentAmt = int(item['sell']['data']['quantity']) * multipliers[sentToken]
        if sentToken not in erc20:
            desc = sentToken + ' #' + item['sell']['data']['token_id']
        if item['sell']['data']['quantity_with_fees'] != '':
            fee = int(item['sell']['data']['quantity_with_fees']) * multipliers[sentToken] - sentAmt
        else:
            fee = 0
        # Incoming token(s):
        tokenAddress = item['buy']['data']['token_address']
        if tokenAddress in tokens:
            recToken = tokens[tokenAddress]
        else:
            recToken = tokenName(tokenAddress)
        recAmt = int(item['buy']['data']['quantity']) * multipliers[recToken]
        if recToken not in erc20:
            desc = recToken + ' #' + item['buy']['data']['token_id']
        
        tstamp = item['timestamp']
        date = tstamp[8:10] + '/' + tstamp[5:7] + '/' + tstamp[0:4] + ' ' + tstamp[11:19]
        if sentToken not in erc20:
            report = report.append({'Date': date, 'Sent Amount':sentAmt, 'Sent Currency': 'NULL', 'Received Amount': recAmt,
                                    'Received Currency': recToken, 'Description': desc, 'TxHash': txId}, ignore_index=True)
        elif recToken not in erc20:
            report = report.append({'Date': date, 'Sent Amount': sentAmt, 'Sent Currency': sentToken, 'Received Amount': recAmt,
                                    'Received Currency': 'NULL', 'Fee Amount': fee, 'Fee Currency': sentToken,
                                    'Description': desc, 'TxHash': txId}, ignore_index=True)
        else:
            report = report.append({'Date': date, 'Sent Amount': sentAmt, 'Sent Currency': sentToken, 'Received Amount': recAmt,
                                    'Received Currency': recToken, 'Fee Amount': fee, 'Fee Currency': sentToken,
                                    'TxHash': txId}, ignore_index=True)
    url = 'https://api.x.immutable.com/v1/orders?user=' + address + '&status=filled&cursor=' + response.json()['cursor']
    response = requests.get(url)

In [ ]:
report

In [ ]:
filename = "Immutable_X_txs_" + address + "_" + dt.strftime(dt.now(), "%Y%m%d_%H%M%S") + ".csv"

In [ ]:
report.to_csv(filename, index=False)